In [1]:
import pyaudio
import wave
import whisper
import os
from fpdf import FPDF

In [2]:
# Configure recording
CHUNK_DURATION = 10
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK_SIZE = 1024
TEMP_FILE = "current_chunk.wav"
OUTPUT_PDF = "Meeting_ZOOM.pdf"

In [3]:
#initialize the whisper 
print ("Loading Whisper model...")
model = whisper.load_model("base")
pdf = FPDF()
pdf.add_page()

Loading Whisper model...


In [4]:
try:
    pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
    pdf.set_font('DejaVu', size=12)
except:
    # Fallback if the path above is different on your ThinkPad
    pdf.set_font("Arial", size=12) 
    print("Warning: Using Arial. Special characters might still cause errors.")


/tmp/ipykernel_14431/2929088518.py:2: DeprecationWarning: "uni" parameter is deprecated since v2.5.1 and will be removed in a future release
  pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)


In [5]:
audio_interface = pyaudio.PyAudio()

ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'


In [6]:
def record_30_second_chunk():
    #record audio for 20 seconds --- specified by CHUNK_DURATION
    stream = audio_interface.open(format=FORMAT,
                                  channels=CHANNELS,
                                  rate=RATE,
                                  input=True,
                                  frames_per_buffer=CHUNK_SIZE)
    frames = []
    for _ in range(0, int(RATE / CHUNK_SIZE * CHUNK_DURATION)):
        data = stream.read(CHUNK_SIZE)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    
    wf = wave.open(TEMP_FILE, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio_interface.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [7]:
print ("Recording... Press Ctrl+C to stop.")

try:
    while True:
        record_30_second_chunk()
        result = model.transcribe(TEMP_FILE, fp16=False)
        text = result['text'].strip()

        if text:
            print(f"Transcribed: {text}")
            pdf.multi_cell(0, 10, text)
            pdf.ln()
except KeyboardInterrupt:
    print("Recording stopped. Saving PDF...")
finally:
    pdf.output(OUTPUT_PDF)
    audio_interface.terminate()


    if os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)    
    print(f"PDF saved as {OUTPUT_PDF}")

Recording... Press Ctrl+C to stop.
Transcribed: as a project and programs manager working with climate works in the UK and a few other
Transcribed: are done by myself together with my core researchers of the priest and it is who you've had from Dr. Perist who you've had from as well and Dr.
Transcribed: and you are basing our research on a study of Ambarata in Madagascar. This is part of an EU funded project called the Repar Project which basically...
Transcribed: parts of South China, Africa across several countries. Marigasca, Niger, Senegal and Ghana being the first ones to
Transcribed: The project has been running for the last two years and is going into the second phase, which is now the installation phase. And as we go through,
Transcribed: we can about the case studies where these projects are being implemented, these being the demo sites and the focus for this particular
Transcribed: Amaratape and Induro Madagascar. And, first of all, just make sense for us to understand why th

Font MPDFAA+DejaVuSansBook is missing the following glyphs: '답' (\ub2f5), '항' (\ud56d), '说' (\u8bf4), '่' (\u0e48), '금' (\uae08)


Recording stopped. Saving PDF...
PDF saved as Meeting_ZOOM.pdf
